---
title: "Download and render astrocyte meshes"
format: 
    html:
        toc: true 
        code-fold: false
jupyter: python3
---

This example uses the plotting package **PyVista** for interactive rendering. Any rendering program that takes vertices and edges can work.

In [1]:
import pandas as pd
import numpy as np

from caveclient import CAVEclient
import pyvista as pv
import cloudvolume 
import seaborn as sns

## Query the proofread astrocytes

In [2]:
# Set version to the most recent datastack with a flat segmentation
client = CAVEclient("minnie65_public")
client.version = 1300

# Load the proofread astrocytes
astrocyte_df = client.materialize.tables.vortex_astrocyte_proofreading_status(status='clean').query(
    split_positions=True)

# sort by y position
astrocyte_df = astrocyte_df.sort_values('pt_position_y')

root_ids = astrocyte_df.pt_root_id.unique()
print(root_ids)

[864691136057675736 864691135852189639 864691135995876778
 864691135527301979 864691135955003656 864691134886294010
 864691135970105957 864691135772920395 864691135384083674
 864691135234576473 864691135164125997 864691136298097435]


## Load the static meshes for all cells

In [3]:
# Set the static segmentaiton source
seg_source = 'precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300'

# load from cloudvolume
cv = cloudvolume.CloudVolume(seg_source, progress=False, use_https=True)
all_meshes = cv.mesh.get(root_ids, lod=3)

## Initialize plotting object, add all astrocytes

In [6]:
pv.set_jupyter_backend("client")
plotter = pv.Plotter()

colormap = list(sns.color_palette('tab20').as_hex())

# Plot the first 6 astrocytes
for mm in range(6):
    mesh = all_meshes[root_ids[mm]]

    # Render the mesh vertices for pyvista
    vertices = mesh.vertices
    faces = mesh.faces
    
    # add a column of all 3s to the faces
    padded_faces = np.concatenate([np.full((faces.shape[0], 1), 3), faces], axis=1)
    
    mesh_poly = pv.PolyData(vertices, faces=padded_faces)
    
    # Flip with y axis
    mesh_poly.points[:, 1] *= -1

    silhouette = dict(
        color=colormap[mm],
        line_width=0.5,
    )

    plotter.add_mesh(mesh_poly, color=colormap[mm], opacity=0.5, silhouette=silhouette)

plotter.camera_position = 'zy'
plotter.camera.zoom(1.0)
plotter.set_background('#fbfbfb')

plotter.show()

Widget(value='<iframe src="http://localhost:54273/index.html?ui=P_0x204afa5cf90_1&reconnect=auto" class="pyvis…

In [5]:
#| echo: false
plotter.export_html('../img/interactives/six_astrocytes.html')


<center> Interactive: Six 'cleaned' astrocytes </center>


<iframe src="../img/interactives/six_astrocytes.html" width="950px" height="600px" style="border: 2px;  "></iframe>